학번: 20150962
이름: 채현종

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
from pathlib import Path
from importlib import reload
from joblib import dump, load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import Callback, EarlyStopping, TensorBoard, ModelCheckpoint

np.random.seed(42)

## 1. CNN에 필요한 Hyper Parameters 설정합니다. 5점
learing_rate = 0.001
traiing_epochs = 5
batch size = 200

In [2]:
learning_rate = 0.001
training_epochs = 5
batch_size = 200

In [3]:
tf.random.set_seed(777)


## 2. 아래의 데이터를 불어옵니다. 5점
returns2020b.csv

In [4]:
returns = pd.read_csv('C:/Users/yssg9187/Desktop/20-2/3. 응용자료분석/D_data/returns2020b.csv')

In [5]:
returns.tail()

,date,1,2,3,4,5,6,7,8,9,...,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11
460555,2020-11-30,3.57,0.70,4.61,0.86,12.38,6.91,-1.88,1.91,-0.87,...,0,0,0,0,0,0,0,0,0,1
460556,2020-11-30,0.64,-1.11,-0.79,0.79,-0.79,1.44,1.46,0.82,0.66,...,0,0,0,0,0,0,0,0,0,1
460557,2020-11-30,0.18,1.09,-0.54,1.84,-0.55,0.18,-0.37,-0.18,0.18,...,0,0,0,0,0,0,0,0,0,1
460558,2020-11-30,-0.17,0.17,-2.37,3.50,-1.89,0.69,-1.37,0.17,-3.47,...,0,0,0,0,0,0,0,0,0,1
460559,2020-11-30,-1.27,-0.50,-5.48,12.90,-3.63,3.76,3.05,0.56,-4.27,...,0,0,0,0,0,0,0,0,0,1


## 3. 자료에 있는 date를 index로 지정합니다. 5점


In [6]:
returns = returns.set_index('date')

## 4. 자료 있는 연도더미를 삭제합니다. 5점

In [7]:
returns = returns.drop([c for c in returns.columns if str(c).startswith('year')], axis=1)

## 5. 2020년 9월까지를 train data, 2020년 10월 11월 데이터를 test data로 지정합니다. 10점


In [8]:
X_train = returns[:'2020-10'].drop('label', axis=1)
y_train = returns[:'2020-10'].label
X_train

,1,2,3,4,5,6,7,8,9,10,...,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11
date,,,,,,,,,,,,,,,,,,,,,
2020-02-10,-0.600000,0.61,-0.30,-0.30,0.61,-0.60,-0.60,-0.30,-2.90,-1.99,...,1,0,0,0,0,0,0,0,0,0
2020-02-10,-0.900000,2.45,-2.68,2.44,1.23,-2.11,-4.06,0.58,-7.30,0.27,...,1,0,0,0,0,0,0,0,0,0
2020-02-10,-1.270000,-2.08,-3.60,-0.27,-0.27,-2.21,-0.65,0.00,-6.17,-1.20,...,1,0,0,0,0,0,0,0,0,0
2020-02-10,0.000000,0.62,-1.67,-1.94,-1.47,-0.44,-1.01,0.29,-1.85,-0.71,...,1,0,0,0,0,0,0,0,0,0
2020-02-10,-1.300000,6.82,0.00,-2.58,-2.90,2.02,1.78,-0.95,-3.78,-0.26,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-29,-10.891681,-0.84,-2.97,2.35,4.78,-3.36,2.00,-4.63,-4.18,5.95,...,0,0,0,0,0,0,0,1,0,0
2020-09-29,3.830000,0.58,-1.14,-0.19,-3.49,-0.55,0.74,-1.98,-1.60,0.71,...,0,0,0,0,0,0,0,1,0,0
2020-09-29,2.240000,-1.88,0.00,2.41,-5.61,0.61,2.34,-3.47,-3.21,0.59,...,0,0,0,0,0,0,0,1,0,0


In [9]:
X_test = returns['2020-10':].drop('label', axis=1)
y_test = returns['2020-10':].label

## 6. CNN을 실행하기 위해 train data와 test data의 차원을 조정합니다. 10점

In [10]:
X_train = X_train.values.reshape(-1, X_train.shape[1], 1)
X_test = X_test.values.reshape(-1, X_train.shape[1], 1)

In [11]:
train_images = np.expand_dims(X_train, axis=-1)
test_images = np.expand_dims(X_test, axis=-1)
#4차원으로 만들기 위해 마지막에 차원 추가, 마지막은 1(흑백)

In [12]:
train_labels = keras.utils.to_categorical(y_train, 2)
test_labels = keras.utils.to_categorical(y_test, 2)    
# one hot encoding으로 만드는 부분

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

## 7. 다음과 같이 CNN 모델을 설정합니다. 10점
conv2d filer 16, kernel_size 2,
maxpooling, padding = same
conv23 filer 32, kernel_size 2
maxpooling, padding = same
fully connected layer 64
layer dropout 0.2
fully connected layer 2

In [14]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=16, kernel_size=2, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(34, 1, 1)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    
    model.add(keras.layers.Conv2D(filters=32, kernel_size=2, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))

    model.add(keras.layers.Flatten()) # vector로 펴줌
    model.add(keras.layers.Dense(64, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(2))
    return model

In [15]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 34, 1, 16)         80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 17, 1, 16)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 1, 32)         2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 1, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 288)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                18496     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0

## 8. Loss Function 설정하기 5점

In [16]:
def loss_fn(model, images, labels):
    logits = model(images, training=True) # TRUE, drop이 적용하게됨
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))    
    return loss 

## 9. Caculating Gradient Function 설정하기 5점

In [17]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

## 10. adam Optimizer 설정 5점

In [18]:
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

## 11. 모델 performance, Caculating Model's Accuracy 함수를 작성합니다. 5점

In [19]:
def evaluate(model, images, labels):
    logits = model(images, training=False) # drop이 실행 안됨
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

## 12. 훈련시키기를 실시하고 최종 test accuracy를 제시합니다. 10점

In [20]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_dataset:
        grads = grad(model, images, labels)                
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:        
        acc = evaluate(model, images, labels)        
        avg_test_acc = avg_test_acc + acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
    #checkpoint.save(file_prefix=checkpoint_prefix)

print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 1 loss = 0.67525756 train accuracy =  0.5683 test accuracy =  0.5477
Epoch: 2 loss = 0.68351817 train accuracy =  0.5640 test accuracy =  0.5478
Epoch: 3 loss = 0.67349446 train accuracy =  0.5730 test accuracy =  0.5518
Epoch: 4 loss = 0.67025393 train accuracy =  0.5579 test accuracy =  0.5500
Epoch: 5 loss = 0.66102898 train accuracy =  0.5622 test accuracy =  0.5482
Learning Finished!


## 13. CNN 결과에 대한 본인의 의견을 기술합니다. 10점

수업시간에 실습하여본 모델보다 테스트 정확도가 소폭 상승하긴 하였지만, 실제 주가 예측에 사용하기에는 힘든 수치이다.